In [36]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
import google.ai.generativelanguage as glm
import textwrap
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import re

In [2]:
load_dotenv()

True

In [3]:
dataset = load_dataset('stanfordnlp/imdb')

In [4]:
# Let's convert them to dataframes
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

In [5]:
df_test.head(10)

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
5,I had high hopes for this one until they chang...,0
6,Isaac Florentine has made some of the best wes...,0
7,"It actually pains me to say it, but this movie...",0
8,"Technically I'am a Van Damme Fan, or I was. th...",0
9,"Honestly awful film, bad editing, awful lighti...",0


In [26]:
genai.configure(api_key=os.environ['GOOGLE_KEY'])
model = genai.GenerativeModel(model_name='gemini-1.0-pro')

In [33]:
reviews = '\n'.join([review for review in df_test.loc[10:20].text])

In [34]:
result = model.generate_content(f"""
    I will be providing upto 10 movie reviews. For each review, find the sentiment and tag it into either positive or negative.
    
    Return an array of the sentiments in the order of the reviews provided, separated by commas.

{reviews}
""")

In [35]:
result.text

Negative, Positive, Negative, Negative, Negative, Negative, Negative, Positive, Negative, Positive


In [49]:
re.search(r'(Negative,?|Positive,?){1,10}', result.text, re.IGNORECASE)

In [61]:
sentiments = [sentiment.lower().strip() for sentiment in result.text.split(',')]
sentiment_nums = []
for sentiment in sentiments:
    print(sentiment)
    if sentiment == 'negative':
        sentiment_nums.append(0)
    else:
        sentiment_nums.append(1)

negative
positive
negative
negative
negative
negative
negative
positive
negative
positive


In [62]:
sentiment_nums

[0, 1, 0, 0, 0, 0, 0, 1, 0, 1]

In [59]:
sentiments

['negative',
 ' positive',
 ' negative',
 ' negative',
 ' negative',
 ' negative',
 ' negative',
 ' positive',
 ' negative',
 ' positive']

In [78]:
res_df_path = '../../data/gemini_sentiment.csv'

In [79]:
res_df = pd.read_csv(res_df_path)

In [80]:
res_df.head(50)

,text,predicted_labels,actual_labels,execution_time
0,I love sci-fi and am willing to put up with a ...,0,0,0.469421
1,"Worth the entertainment value of a rental, esp...",0,0,0.469421
2,its a totally average film with a few semi-alr...,0,0,0.469421
3,STAR RATING: ***** Saturday Night **** Friday ...,1,0,0.469421
4,"First off let me say, If you haven't enjoyed a...",0,0,0.469421
5,I had high hopes for this one until they chang...,0,0,0.469421
6,Isaac Florentine has made some of the best wes...,0,0,0.469421
7,"It actually pains me to say it, but this movie...",0,0,0.469421
8,"Technically I'am a Van Damme Fan, or I was. th...",0,0,0.469421
9,"Honestly awful film, bad editing, awful lighti...",0,0,0.469421
